In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import gc

%load_ext autoreload
%autoreload 2

In [5]:
# прописывем нужные нам пути
dir_project = 'git/proj_news_viz/nlp' # путь проекта

PATH = os.path.join(os.getenv('HOME'), dir_project)
sys.path.append(PATH)

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel, LdaMulticore

---

In [6]:
df = pd.read_csv(f'{PATH}/data/interim/news_lenta-cleaned-lemmatize.csv.gz', compression='gzip')
print(df.shape)
df.sample(2)

(674580, 2)


,url,lemma_text
565114,https://lenta.ru/world/2005/01/06/resign/,мужской сборная россия занять третье место эст...
74275,https://lenta.ru/news/2017/01/13/shadow_brokers/,канадский премьера осудить путешествие вертолё...


In [42]:
# преобразуем текст в список токенов
df['lemma_text_list'] = df.lemma_text.apply(lambda x: x.split())
df.sample(2)

,url,lemma_text,lemma_text_list
629957,https://lenta.ru/news/2002/02/25/lenfilm/,тягачев россия всё-таки участвовать церемония ...,"[тягачев, россия, всё-таки, участвовать, церем..."
85458,https://lenta.ru/news/2016/11/07/sex_robots/,уличить согласование вопрос трамп демократ сай...,"[уличить, согласование, вопрос, трамп, демокра..."


In [29]:
# проверим и удалим пустые записи и дубликаты
df.dropna(inplace=True)
df.lemma_text.drop_duplicates(inplace=True)

print(f'df.shape = {df.shape}')
df.isnull().sum()

df.shape = (674580, 2)


url           0
lemma_text    0
dtype: int64

---

### LDA

Для работы с LDA нам нужно подготовить 2 объекта:

1. словарь, в котором каждому слову присваивается уникальный id

In [43]:
%%time
dictionary = corpora.Dictionary(df.lemma_text_list)

CPU times: user 1min 57s, sys: 15.5 s, total: 2min 13s
Wall time: 2min 22s


In [47]:
len(dictionary)

460587

In [59]:
dictionary[0]

'ансворт'

2. список преобразованных текстов ("мешок слов" или tf-idf)

In [50]:
%%time
corpus = [dictionary.doc2bow(doc) for doc in df.lemma_text_list]

CPU times: user 1min 44s, sys: 1min 25s, total: 3min 10s
Wall time: 3min 41s


In [82]:
print(corpus[:1])

[[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 3), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)]]


In [84]:
# или можем посмотреть более человеческим взглядом
print([[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]])

[[('ансворт', 2), ('британский', 2), ('верна', 1), ('видеть', 1), ('внутри', 1), ('вскоре', 1), ('вход', 2), ('вызволять', 1), ('глубокий', 1), ('грот', 1), ('дайвера', 2), ('день', 1), ('десять', 1), ('детский', 1), ('дождь', 1), ('живой', 1), ('завалить', 1), ('засунуть', 1), ('затопить', 3), ('заявить', 1), ('идея', 1), ('илона', 2), ('инцидент', 1), ('исчезновение', 1), ('июль', 1), ('июнь', 1), ('камень', 1), ('километр', 1), ('команда', 1), ('крутой', 1), ('маск', 3), ('маска', 1), ('место', 1), ('миллиардер', 1), ('мини-субмарина', 1), ('морпех', 1), ('назвать', 2), ('найти', 1), ('один', 1), ('операция', 1), ('оппонент', 1), ('ответ', 1), ('ответить', 1), ('отделять', 1), ('отметить', 1), ('педофил', 2), ('пещера', 3), ('пиар-ход', 1), ('план', 1), ('по-настоящему', 1), ('подлодка', 1), ('поздний', 1), ('пойти', 1), ('посоветовать', 1), ('пошлый', 1), ('пропасть', 1), ('раскритиковать', 1), ('ребёнок', 3), ('резка', 1), ('свобода', 1), ('свой', 1), ('сидеть', 1), ('сначала', 1)